In [2]:
# gmailer init


In [260]:
import datetime as dt

import yaml

import requests
import smtplib
import ssl
import mimetypes
from email.message import EmailMessage

import pandas as pd



## Load Configurations

In [298]:
## password and config
config = "../config.yaml"
config = yaml.safe_load(open(config))

def parse_config(config: dict):
    """
    Parse the configuration yaml.

    Parameters
    -------
    config (dict{any}): Dictionary of congiruations.

    Returns
    -------
    credentials (dict(any)): Dictionary of credentials.
    """
    credentials = config["credentials"]
    ...
    return credentials

credentials = parse_config(config)

# Methods

In [262]:
def get_csv_as_list(filepath: str) -> list:
    """
    Get list using a path to a csv.

    Parameters
    -------
    filepath (str): filepath. 

    Returns
    -------
    ([str]): List of values.
    
    """
    import pandas as pd
    recipients = pd.read_csv(recipients_csv_path, header=None)
    return recipients[0].values.tolist()


#### Potential Class 0
class Clearbit():
    """A class for working with Clearbit API."""
    def get_name(response: requests.models.Response) -> (str, str):
        """
        Parse response for HTTP request.

        Parameters
        -------
        response (requests.models.Response): Http response with person data.

        Returns
        -------
        name (str, str): Tuple of first and last name. 
        """
        if not isinstance(y, requests.models.Response):
            print("Not a request.")
            return 
        name = (None, None)
        data = response.json()
        person = data["person"]
        first_name = person["name"]["givenName"]
        last_name = person["name"]["familyName"]
        name = (first_name, last_name)
        return name


    def get_names_from_email_list(recipients_list:[str], username=None, password=None, api_key=None):
        """
        Given a list of recipients, use Clearbit to retreive their names. Other data may be retreieved but at a later stage. 
        The username is the api_key from Clearbit. Read their docs for more info.

        Parameters
        -------
        username (str): Optional. Username.
        password (str): Optional. Password.
        api_key (str): API key.

        Returns
        -------
        names {}: Dict of emails to names.
        """
        names = {}
        # NOTE: may want to batch this in the future or too many requests will be attempted too quickly.
        for email in recipients_list:
            url = f"https://person.clearbit.com/v2/combined/find?email=:{email}"
            clearbit_response = get_response(url, username=clearbit_api_key, password=None, api_key=None)
            names[email] = get_name(clearbit_response)
        return names
#####


def get_response(url: str, username=None, password=None, api_key=None):
    """
    Get a response from API using HTTP.

    Parameters
    -------
    url (str): Url for API request.
    username (str): Optional. Username.
    password (str): Optional. Password.
    api_key (str): API key.

    Returns
    -------
    api_response (requests.models.Response): Response.
    """
    api_response = None
    api_response = requests.get(url, auth=(username, password))
    return api_response





# Email class

In [263]:
sender = "jaime.meriz13@gmail.com"

# recipient path should be: list of recipients. Single recipient.
# recipients_csv_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/email_recipients.csv"
recipients_csv_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/email_recipients_2.csv"
recipients = get_csv_as_list(recipients_csv_path)

subject = "DEV" + "-" "Jmail" + "-" + str(dt.date.today())
body = "body\nyours\nDr. Lopez"
# password = "password"

In [131]:
def send_mail(sender: str, recipients: list, subject:str, body: str, password: str, attachment_path=None):
    """
    Send an email via SMTP.

    Parameters
    -------
    sender (str): Sender as a string.
    recipients ([str]): List of recipients. 
    subject (str): Email subject line.
    body (str): Email body as string.
    password (str): Gmail password as string.
    attachment_path (str): Optional. Filepath to attachment. 
    
    Returns
    -------
    None
    """
    def add_attachment(email: EmailMessage, filepath: str) -> EmailMessage:
        """
        Given original email message. May or may not include an attachment already.

        Parameters:
        -------
        email (EmailMessage): EmailMessage.
        filepath (str): Filepath to attachment.

        Returns
        -------
        email (EmailMessage): Mail message with new attachment.
        """
        # Attachments
        with open(filepath, "rb") as fp:
            data = fp.read()

        # guess encoding
        ctype, encoding = mimetypes.guess_type(filepath)
        if ctype is None or encoding is not None:
            # No guess could be made, or the file is encoded (compressed), so
            # use a generic bag-of-bits type.
            ctype = "application/octet-stream"
        maintype, subtype = ctype.split("/", 1)

        email.add_attachment(data, maintype=maintype, subtype=subtype)
        print(f"Successfully attached: {filepath}")
        return email


    email = EmailMessage()
    email["Sender"] = sender
    email["Recipients"] = ", ".join(recipients)
    email["Subject"] = subject
    email.set_content(body)
    if filepath != None:
        email = add_attachment(email, filepath)

    # Email failsafe.
    confirm_send = input(f"Are you sure you want to send to recipients? (Y/N) \n\n {recipients}\n")

    if confirm_send.lower() == "y":
        context = ssl.create_default_context()
        print("Sending email...")
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp_server:
            smtp_server.login(sender, password)
            smtp_server.sendmail(sender, recipients, email.as_string())
        print("Message sent!")
    else:
        print("Message NOT sent!")


    return email

password = credentials["gmail"]["app_password"]
email = send_mail(sender, recipients, subject, body, password)


Successfully attached: SF_Ryu.png
Sending email...
Message sent!


# Scraping / Using Selenium

In [265]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

BROWSER_STALL_SECONDS = 1

opts = Options()
# opts.add_argument("--headless")
# browser = Chrome(options=opts)
browser = Chrome()

# try browsing to your gmail account.
url = "https://mail.google.com/mail/u/0/#inbox"
browser.get(url)

# 
gmailId = "jaime.meriz13@gmail.com"
browser.implicitly_wait(BROWSER_STALL_SECONDS)
login_box = browser.find_element("xpath", '//*[@id ="identifierId"]')
login_box.send_keys(gmailId)


next_button = browser.find_element("xpath", '//*[@id ="identifierNext"]')
next_button.click()


# Looks like 
password = credentials["gmail"]["password"]
browser.implicitly_wait(BROWSER_STALL_SECONDS)
password_box = browser.find_element("xpath", '//*[@name="Passwd"]') 

password_box.send_keys(password)

# nextButton = browser.find_element("xpath", '//*[@id ="passwordNext"]')
# nextButton.click()


In [256]:
y = browser.find_element("xpath", '//*[@id ="next"]')
y.get_attribute("jsname")
# This confirms that we are being redirected to an error page so we need to fix that first and
# trick chrome into thinking we have a secure browser.


# Try undedected chrome driver

In [335]:
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options


# Constants
BROWSER_STALL_SECONDS = 1


# Starting the driver.
options = Options()
# options.page_load_strategy = "eager"
args = ['--disable-web-security', '--user-data-dir', '--allow-running-insecure-content']
for arg in args:
    options.add_argument(arg)

browser = uc.Chrome(options=options, headless=False, use_subprocess=False)


# Navigating.
url = "https://mail.google.com/mail/u/0/#inbox"
browser.get(url)

# Accessing Gmail.
# User login
gmailId = "jaime.meriz13@gmail.com"
browser.implicitly_wait(BROWSER_STALL_SECONDS)
login_box = browser.find_element("xpath", '//*[@id ="identifierId"]')
login_box.send_keys(gmailId)

# Submit and navigate.
next_button = browser.find_element("xpath", '//*[@id ="identifierNext"]')
next_button.click()

# User password
password = credentials["gmail"]["password"]
browser.implicitly_wait(BROWSER_STALL_SECONDS)
password_box = browser.find_element("xpath", '//*[@name="Passwd"]') 
password_box.send_keys(password)

# Submit and navigate.
nextButton = browser.find_element("xpath", '//*[@id="passwordNext"]')
nextButton.click()


# browser.close()


In [331]:
# z = browser.find_element("xpath", '//*[@id=":1e"]')
z

<undetected_chromedriver.webelement.WebElement (session="122a6bff71536e24ad43b8a8898b1a5e", element="25350B54980EAC2F530ECDCD7DF99209_element_422")>

It's mostly working with the exception that because of the 2-factor verification process, we're being blocked. Maybe I can turn it off?  

Yes, successfully turned this off!